In [1]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
from nwbwidgets.allen import AllenRasterWidget
from nwbwidgets.allen import AllenPSTHWidget
import pandas as pd
import numpy as np
from helpers import *


# Load the file with the mice data
nwbfiles = []

paths = ['/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839.nwb', 
        '/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB087_20231017_141901.nwb']
mouse_names = ['AB077_20230531_143839', 'AB087_20231017_141901']

for path in paths:
    io = NWBHDF5IO(path)
    nwbfiles.append(io.read())

io = NWBHDF5IO('/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839.nwb', mode='r')
nwbfile = io.read()
# To do : create a list of mouse names, to be used for the .py file.
mouse_name = 'AB077_20230531_143839'

/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


### Loading the tables and preprocessing

#### For each line of trials table, we start by testing 10 neurons. In other words, if there is a stimulus (whisker or auditory) at a certain start time then, compute the number of spikes for each trial for each neuron.

In [2]:
units = nwbfile.units.to_dataframe()
trials = nwbfile.trials.to_dataframe()

# Preprocessing s
filtered_units = units[(units['bc_label'] == 'good') & (units['ccf_acronym'].str.contains('[A-Z]'))]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 10)  # Show all columns


# Some chosen columns
cons_columns  = ["cluster_id", "firing_rate", "ccf_acronym", "ccf_name", "ccf_parent_acronym", "ccf_parent_name", "spike_times"]#, "lick_flag", "lick_time"]
filtered_units = filtered_units[cons_columns]

In [10]:
units

,cluster_id,peak_channel,electrode_group,depth,ks_label,group,bc_label,firing_rate,maxChannels,bc_cluster_id,useTheseTimesStart,useTheseTimesStop,percentageSpikesMissing_gaussian,percentageSpikesMissing_symmetric,presenceRatio,nSpikes,nPeaks,nTroughs,isSomatic,waveformDuration_peakTrough,spatialDecaySlope,waveformBaselineFlatness,rawAmplitude,signalToNoiseRatio,fractionRPVs_estimatedTauR,waveform_mean,sampling_rate,duration,pt_ratio,ccf_ml,ccf_ap,ccf_dv,ccf_id,ccf_acronym,ccf_name,ccf_parent_id,ccf_parent_acronym,ccf_parent_name,spike_times
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,imec1_shank0 pynwb.ecephys.ElectrodeGroup at 0...,20.0,good,,non-soma,3.6338799,1.0,1.0,1440.0029666666667,3240.0029666666665,1.5899386555672135,5.544531341192949,1.0,8092.0,2.0,1.0,0.0,566.6666666666666,-0.0185412143275834,0.020620175216435287,87.79679694387306,84.79378458594697,0.02219452999432176,"[-82.3885719658977, -82.14173771313942, -81.55...",30000,0.506158023968649,0.3315226131724009,2824.2160930484556,4603.321407960273,5102.370572535306,772,ACAv5,"Anterior cingulate area, ventral part, layer 5",48.0,ACAv,"Anterior cingulate area, ventral part","[0.29488899999999996, 0.31685566666666665, 3.8..."
1,1,1,imec1_shank0 pynwb.ecephys.ElectrodeGroup at 0...,20.0,mua,,good,4.97110188,2.0,2.0,1800.0029666666667,3240.0029666666665,9.560013900127373,25.90602804589328,1.0,8137.0,1.0,1.0,1.0,733.3333333333334,-0.017688404011155723,0.04405705262996807,43.31432181979952,42.5555178186307,0.02344297067914623,"[-104.53125, -104.45517015706807, -104.7104057...",30000,0.5122916565035694,0.548282149369403,2824.2160930484556,4603.321407960273,5102.370572535306,772,ACAv5,"Anterior cingulate area, ventral part, layer 5",48.0,ACAv,"Anterior cingulate area, ventral part","[0.479889, 1.1120556666666668, 2.586543, 4.527..."
2,2,1,imec1_shank0 pynwb.ecephys.ElectrodeGroup at 0...,20.0,good,,non-soma,3.01005127,2.0,3.0,0.0029666666666666665,1080.0029666666667,5.792532698010722,23.609845031905195,1.0,4190.0,1.0,1.0,0.0,166.66666666666666,-0.01506102721493984,0.005711781504124372,43.18557793164528,22.2860202240289,0.0,"[-104.72234828838174, -104.75638615145229, -10...",30000,0.7489698684488675,0.4344907660953186,2824.2160930484556,4603.321407960273,5102.370572535306,772,ACAv5,"Anterior cingulate area, ventral part, layer 5",48.0,ACAv,"Anterior cingulate area, ventral part","[0.2628556666666667, 0.4440556666666667, 2.131..."
3,3,2,imec1_shank0 pynwb.ecephys.ElectrodeGroup at 0...,40.0,mua,,good,0.14528229,3.0,4.0,0.0029666666666666665,3240.0029666666665,2.352659466438034,0.0,1.0,526.0,1.0,1.0,1.0,433.3333333333333,-0.023903861631071176,0.020465062550861366,157.65576559285924,86.67874662372151,0.0,"[-98.50050403225806, -98.13088037634408, -97.5...",30000,0.4244207034489488,0.2684342184269639,2814.844399717983,4597.780031667841,5103.663374443018,772,ACAv5,"Anterior cingulate area, ventral part, layer 5",48.0,ACAv,"Anterior cingulate area, ventral part","[7.349212333333333, 36.45573866666666, 56.7888..."
4,4,2,imec1_shank0 pynwb.ecephys.ElectrodeGroup at 0...,40.0,mua,,good,3.30061586,3.0,5.0,0.0029666666666666665,720.0029666666667,9.96600994348007,12.495266944339269,1.0,2311.0,1.0,1.0,1.0,566.6666666666666,-0.015218920456357069,0.03384722083853709,50.613188271378185,56.084961719601374,0.03683391019262747,"[-97.21519595286885, -97.12154200819673, -97.2...",30000,0.5802949737385562,0.5987893571810807,2814.844399717983,4597.780031667841,5103.663374443018,772,ACAv5,"Anterior cingulate area, ventral part, layer 5",48.0,ACAv,"Anterior cingulate area, ventral part","[0.680589, 2.2582096666666667, 2.8338096666666..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,724,310,imec2_shank0 pynwb.ecephys.ElectrodeGroup at 0...,3120.0,mua,,mua,0.10518606,293.0,725.0,0.0027,2880.0027,4.328173744230024,23.404255319148945,0.8958333333333334,144.0,1.0,1.0,1.0,566.666666666

### Whisker &  Stimulation
#### Now iterate in each row of short\_units and create a column with tables to append values for number of spikes for each 

In [9]:
trials['context_background'].unique()

array([nan])

In [ ]:
### part 1 : with passive trials
active_trials = trials[trials['context']] == 'passive'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
### part 2 : with active trials
passive_trials = trials[trials['context']] == 'active'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
# Create a new dataframe 

data['pre'] = 0.2
data['post'] = 0.2
data['Mouse name'] = mouse_name



folder = f'Data/{mouse_name}'
os.makedirs(folder, exist_ok=True)

#### Only problem : arrays present in columns are converted to string of array, so careful while preprocessing ####

###### It is better to use parquet files ######
#data.to_csv(f'{mouse_name}_Selectivity_Dataframe.csv', index=True, header=True)
#data.to_pickle(f'{mouse_name}_Selectivity_Dataframe.pkl')
data.to_parquet(f'{folder}/{mouse_name}_Selectivity_Dataframe.parquet', index=False)